# Implementation of Vanilla GANs model
Reference: https://arxiv.org/pdf/1406.2661.pdf

Fuente: https://github.com/Yangyangii/GAN-Tutorial


In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image

In [3]:
import numpy as np
import datetime
import os, sys

In [4]:
from matplotlib.pyplot import imshow, imsave
%matplotlib inline

In [5]:
from google.colab import drive
drive.mount("/content/drive")
data_dir = '/content/drive/My Drive/CIA_marcos/deep_learning/clase_8/GANs'

Mounted at /content/drive


In [5]:
MODEL_NAME = 'VanillaGAN'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# defino el tamaño del vector del ruido del cual deseo generar a imagen.
n_noise = 100

In [7]:
class Generator(nn.Module):
    """
        Simple Generator w/ MLP
    """
    def __init__(self, input_size=n_noise, num_classes=784): # input_size lo elijo yo,
                                                             # es el vector de números aleatorios del cual deseo partir.
        super(Generator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, num_classes),
            nn.Tanh()
        )

    def forward(self, x):
        y_ = self.layer(x)
        y_ = y_.view(x.size(0), 1, 28, 28) # la salida le doy formato de imagen de MNIST [batchx1x28x28]
        return y_

In [15]:
class Discriminator(nn.Module):
    """
        Simple Discriminator w/ MLP
    """
    def __init__(self, input_size=784, num_classes=1): # input_size, la imagen MNIST vectorizada, num_classes=1 (REAL or FAKE)
        super(Discriminator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, num_classes),
            nn.Sigmoid(),
        )

    def forward(self, x):
        y_ = x.view(x.size(0), -1) # vectorizo la imagen MNIST
        y_ = self.layer(y_)
        return y_

In [8]:
def get_sample_image(G, n_noise):
    """
        save sample 100 images
    """
    z = torch.randn(100, n_noise).to(DEVICE) # genero un batch de 100 vectores randn
                                            # del tamaño definido en n_noise.
    # le paso ese batch al Generador y reordeno la salida que me da.
    y_hat = G(z).view(100, 28, 28) # (100, 28, 28)
    result = y_hat.cpu().data.numpy()

    # preparo array para guardar los 100 resultados del generador
    img = np.zeros([280, 280])
    for j in range(10):
        img[j*28:(j+1)*28] = np.concatenate([x for x in result[j*10:(j+1)*10]], axis=-1)
    return img

In [16]:
# instancio los modelos
D = Discriminator().to(DEVICE)
G = Generator(n_noise).to(DEVICE)


In [12]:
# puedo cargar un modelo ya entrenado
D.load_state_dict(torch.load(data_dir + '/D_nnoise_100_e_100.pkl', map_location=DEVICE, weights_only=False))


<All keys matched successfully>

In [13]:
G.load_state_dict(torch.load(data_dir + '/G_nnoise_100_e_100.pkl', map_location=DEVICE, weights_only=False))

<All keys matched successfully>

In [17]:
# transformación para normalizar el dataset.
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5],
                                std=[0.5])]
)

In [18]:
mnist = datasets.MNIST(root='../data/', train=True, transform=transform, download=True)

In [19]:
batch_size = 32 #32 original

In [20]:
data_loader = DataLoader(dataset=mnist, batch_size=batch_size, shuffle=True, drop_last=True)

In [21]:
criterion = nn.BCELoss()
D_opt = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
G_opt = torch.optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [22]:
max_epoch = 30 # need more than 10 epochs for training generator
step = 0
n_critic = 1 # for training more k steps about Discriminator

In [23]:
D_true = torch.ones(batch_size, 1).to(DEVICE) # Discriminator Label to real
D_fakes = torch.zeros(batch_size, 1).to(DEVICE) # Discriminator Label to fake

In [24]:
if not os.path.exists('samples'):
    os.makedirs('samples')

In [25]:
step = 0
for epoch in range(max_epoch):
    for idx, (images, _) in enumerate(data_loader):
        # Training Discriminator
        # el discriminador debe ser entenado para poder reconocer imagenes
        # REAL vs FAKE

        # le paso imagenes de verdad y etiquetas de verdad.
        x = images.to(DEVICE)
        x_outputs = D(x)
        D_x_loss = criterion(x_outputs, D_true)

        # le paso imágenes falsas y etiquetas falsas.
        z = torch.randn(batch_size, n_noise).to(DEVICE) # vector randn
        fake_im = G(z) # genero con el generador
        z_outputs = D(fake_im) # las paso por el Discriminador
        D_z_loss = criterion(z_outputs, D_fakes) # comparo con etiquetas FAKE

        # perdida total discriminador
        D_loss = D_x_loss + D_z_loss # sumo ambas pérdidas

        # optimizo el Discriminador
        D.zero_grad()
        D_loss.backward()
        D_opt.step()


        if step % n_critic == 0:
            # Training Generator
            # el Generador, debe ser entrenado para engañar al Discriminador!
            # para esto, le paso imágenes falsas al Discriminador, CON ETIQUETAS REALES!
            z = torch.randn(batch_size, n_noise).to(DEVICE)
            z_outputs = D(G(z))
            G_loss = criterion(z_outputs, D_true)

            # optimizo SOLO el Generador
            G.zero_grad()
            G_loss.backward()
            G_opt.step()

        # lineas plara sacar resultados parciales o imágenes parciales
        if step % 500 == 0:
            print('Epoch: {}/{}, Step: {}, D Loss: {}, G Loss: {}'.format(epoch, max_epoch, step, D_loss.item(), G_loss.item()))

        if step % 1000 == 0:
            G.eval()
            img = get_sample_image(G, n_noise)
            imsave('samples/{}_step{}.jpg'.format(MODEL_NAME, str(step).zfill(3)), img, cmap='gray')
            G.train()
        step += 1

Epoch: 0/30, Step: 0, D Loss: 1.4519517421722412, G Loss: 0.7058539390563965
Epoch: 0/30, Step: 500, D Loss: 1.3584978580474854, G Loss: 1.1535279750823975
Epoch: 0/30, Step: 1000, D Loss: 1.2303850650787354, G Loss: 0.8293906450271606
Epoch: 0/30, Step: 1500, D Loss: 1.1970837116241455, G Loss: 0.821012020111084
Epoch: 1/30, Step: 2000, D Loss: 1.218174695968628, G Loss: 0.9845958948135376
Epoch: 1/30, Step: 2500, D Loss: 1.3657243251800537, G Loss: 0.5412221550941467
Epoch: 1/30, Step: 3000, D Loss: 1.0821744203567505, G Loss: 0.8686199188232422
Epoch: 1/30, Step: 3500, D Loss: 1.2104132175445557, G Loss: 1.1827995777130127
Epoch: 2/30, Step: 4000, D Loss: 1.2539044618606567, G Loss: 0.7879659533500671
Epoch: 2/30, Step: 4500, D Loss: 1.1937105655670166, G Loss: 1.0218042135238647
Epoch: 2/30, Step: 5000, D Loss: 1.1235241889953613, G Loss: 1.165374994277954
Epoch: 2/30, Step: 5500, D Loss: 1.147237777709961, G Loss: 1.0454576015472412
Epoch: 3/30, Step: 6000, D Loss: 1.2765269279479

KeyboardInterrupt: 

## Visualize Sample

In [ ]:
# generation to image
G.eval()
imshow(get_sample_image(G, n_noise), cmap='gray')

In [ ]:
# Saving params.
#torch.save(D.state_dict(), data_dir + '/D_nnoise_100_e_100.pkl')
#torch.save(G.state_dict(), data_dir + '/G_nnoise_100_e_100.pkl')

In [ ]:
def get_sample_image_col(G, n_noise, z):
    """
        save sample 100 images
    """

    # lo repito 100 veces para generas 100 imagenes
    z2 =  z.repeat(100, 1)

    # defino un spam de valores para reemplazar en una columna
    # recuerdo que mi generador es una randn (distribución normal con media=0 y varianza=1)
    # por lo que en -3 a +3 tendré el 99.9% de los valores que pude haber visto el G
    a = torch.linspace(-3,3,100)

    # elejir que columna deseo reemplazar
    # la cambio a azar
    n = np.random.randint(n_noise)
    print(f'Columna variada: {n}')
    #n = 92
    z2[:,n] = a

    # le paso ese batch al Generador y reordeno la salida que me da.
    y_hat = G(z2).view(100, 28, 28) # (100, 28, 28)
    result = y_hat.cpu().data.numpy()

    # preparo array para guardar los 100 resultados del generador
    img = np.zeros([280, 280])
    for j in range(10):
        img[j*28:(j+1)*28] = np.concatenate([x for x in result[j*10:(j+1)*10]], axis=-1)
    return img


In [ ]:
n_noise

In [ ]:
# genero un batch de 1 vector randn del tamaño definido en n_noise que voy a utilizar en lo que sigue
z = torch.randn(1, n_noise).to(DEVICE)

In [ ]:
z

In [ ]:
imshow(get_sample_image_col(G, n_noise, z), cmap='gray')

In [ ]:
z